# Classificador Naive Bayes

___


___
Carregando algumas bibliotecas:

In [480]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

from scipy import stats
import math

# Para ter melhor print
from IPython.display import display

___
## Índice

- [Questão 1](#q1)
    - [Item a](#q1_a)
    - [Item b](#q1_b)
    - [Item c](#q1_c)
- [Questão 2](#q2)
    - [Item a](#q2_a)
    - [Item b](#q2_b)
    - [Item c](#q2_c)
___

<div id="q1"></div>

___

# Questão 1

O serviço de entregas de comida iQueMedo entrega apenas 4 itens diferentes durante a semana: coxinha (C), carne misteriosa (M), peixe caramelizado - pensa uma maçã do amor, mas de sardinha - (P) e ostra *vintage* (O). Aos sábados servem pizza ou sushi.

O *DataFrame* abaixo mostra o histórico de compras de alguns clientes durante a semana, e qual o pedido que fizeram no sábado. A coluna "Semana" contém uma *string* onde cada letra representa um pedido (dentre as opções acima: 'C', 'M', 'P' e 'O'), e a coluna "Final" mostra o que foi pedido no final de semana.

In [481]:
import os

filename = 'comidas.csv'
if filename in os.listdir():
    print(f'Encontrei o arquivo {filename}, tudo certo para prosseguir com a prova!')
else:
    print(f'Não encontrei o arquivo {filename} aqui no diretório {os.getcwd()}, '
          'será que você não baixou o arquivo?')

Encontrei o arquivo comidas.csv, tudo certo para prosseguir com a prova!


In [482]:
comidas = pd.read_csv(filename, sep=";")

In [483]:
comidas.head()

,Sábado,Semana
0,pizza,PMPCPCC
1,sushi,CPO
2,sushi,PPMPOPOOPP
3,sushi,POPPO
4,pizza,CPMM


<div id="q1_a"></div>

---

**a)** Queremos um sistema que permita escolher, baseado no histórico de pedidos de um cliente, se ele vai preferir sushi ou pizza no sábado.

Construa um classificador Naive-Bayes para esse problema. Ou seja, determine:

- as probabilidades condicionais de comida por preferência de final de semana, e 
- as probabilidades *a priori* das preferências de final de semana.

Observação: você deve adotar as seguintes medidas simplificadoras:

- Não precisa separar em treinamento e teste, estamos interessados apenas no classificador treinado com o conjunto completo de dados.
- Não precisa implementar *smoothing*.
- Não tem "palavra desconhecida": os pedidos são apenas as letras C, M, P e O.

Dica: para separar uma *string* em lista de letras é simples: basta usar a função `list()`. Por exemplo:


In [484]:
texto = 'PPMPOPOOPP'
lista = list(texto)
print(lista)

['P', 'P', 'M', 'P', 'O', 'P', 'O', 'O', 'P', 'P']


In [485]:
pizza = comidas.loc[comidas['Sábado'] == 'pizza', ['Semana']]
sushi = comidas.loc[comidas['Sábado'] == 'sushi', ['Semana']]

In [486]:
pizza = pizza.reset_index(drop=True)
sushi = sushi.reset_index(drop=True)

pedidos_pizza = []
for i in range(len(pizza['Semana'])):
    pedidos_pizza.extend(list(pizza['Semana'][i]))

pedidos_sushi = []
for i in range(len(sushi['Semana'])):
    pedidos_sushi.extend(list(sushi['Semana'][i]))

In [487]:
tabela_pizza_relativo = pd.DataFrame(pedidos_pizza).value_counts(normalize=True)
tabela_sushi_relativo = pd.DataFrame(pedidos_sushi).value_counts(normalize=True)

In [488]:
P_sushi = comidas.Sábado.value_counts()['sushi'] / len(comidas)
P_pizza = comidas.Sábado.value_counts()['sushi'] / len(comidas)

--- 

<div id="q1_b"></div>


**b)** Determine a recomendação do sistema para um cliente com o seguinte histórico de pedidos: "PPCPCCCOPP"

Observação: não precisa implementar o "truque do log".

In [489]:
pedidos = 'PPCPCCCOPP'
pedidos = list(pedidos)

probPedidoDadoPizza = 1
probPedidoDadoSushi = 1

for pedido in pedidos:
    probPedidoDadoPizza *= tabela_pizza_relativo[pedido]
    probPedidoDadoSushi *= tabela_sushi_relativo[pedido]

In [490]:
probPizzaDadoPedido = probPedidoDadoPizza * P_pizza
probSushiDadoPedido = probPedidoDadoSushi * P_sushi

print(probPizzaDadoPedido)
print(probSushiDadoPedido)

5.324807152059935e-07
6.360954843505357e-07


<div id="q1_c"></div>

---

**c)** Saiu uma notícia no jornal que diz que comer peixe caramelizado aumenta muito a incidência de unha encravada, daquelas bem horríveis. (Caso você goste de unha encravada: suponha que a população em geral não gosta.)

Comente o que você espera que aconteça se você continuar usando seu classificador Naive-Bayes para fazer as recomendações de pizza ou sushi.

(Nesta questão não tem código ou cálculos para fazer - apenas senso crítico.)

In [491]:
# Coloque sua resposta aqui...

---

<div id="q2"></div>

# Questão 2 - análise de texto

___

![Fonte: https://commons.wikimedia.org/wiki/File:CocaColaBottle_background_free.png](img/160px-CocaColaBottle_background_Free.png)

Esta questão é baseada no Projeto 2

Você foi contratado pela empresa *The Coca Cola Company* para fazer monitoramento de redes sociais. Sua empresa fez um *bot* que continuamente busca por novas postagens no Twitter e as classifica.

A empresa está interessada em saber quais mensagens estão felizes com a marca versus o total que estão neutros ou insatisfeitos.

Sua base de treinamento está no arquivo `coca_cola.xlsx`.  

As mensagens felizes têm a coluna **Categoria** com valor `FELIZ` e as mensagens que não são felizes têm como **Categoria** o valor `NAO_FELIZ`.

Note que a coluna Texto já passou pelas seguintes etapas:
* Conversão para letras minúsculas
* Limpeza de pontuação

In [492]:
tweets = pd.read_excel("Q3_coca_cola.xlsx")

In [493]:
tweets.head()

,Texto,Categoria
0,acho que comer hamburguer de soja e se entupir...,NAO_FELIZ
1,gosto mais de vaca preta coca cola com sorvet...,FELIZ
2,me prometo uma coca cola gelada para o fim do...,FELIZ
3,essa coca cola gelada fez meu dia,FELIZ
4,prometo parar com doce refrigerante coca cola ...,NAO_FELIZ


Pede-se:

<div id="q2_a"></div>

---

**a)** Junte todas as mensagens da cada categoria em uma *string* única


In [494]:
feliz = tweets.loc[tweets['Categoria'] == 'FELIZ', ['Texto']]
nao_feliz = tweets.loc[tweets['Categoria'] == 'NAO_FELIZ', ['Texto']]

In [495]:
tweets_string = ''

for i in range(len(tweets['Texto'])):
    tweets_string += tweets['Texto'][i] + ' '

tweets_string

'acho que comer hamburguer de soja e se entupir de coca cola é ser hipócrita gosto mais de vaca preta  coca cola com sorvete  do que de milk shake me prometo uma coca cola gelada para o  fim do dia essa coca cola gelada fez meu dia prometo parar com doce refrigerante coca cola etc maior bronca da nutricionista tomei tanta coca cola que zoei meu rim a combinação de sushi e cocacola é uma maravilha\n\nfelizmente o sushi nem tá perto de acabar miltonneves já chegou coca cola em muzambinho deixei a coca na geladeira e virou uma espécie de picolé pena que ficou preso na garrafa meu ser precisa de uma coca cola agora matei a última coca cola que tinha na geladeira 😊 esse ritmo de 6l de coca cola por dia me deixou com diabetes coca cola é uma porcaria açucarada vai se f big mac com coca cola eu mereço a coca cola patrocina esportes para nos fazer achar que engordamos porque não fazemos exercício num dia quente assim quero me entupir de cheetos e coca cola eu sou apaixonada em coca cola 😍😍😍😍😰 

In [496]:
string_feliz = ''
feliz = feliz.reset_index(drop=True)

for i in range(len(feliz['Texto'])):
    string_feliz += feliz['Texto'][i] + ' '

string_feliz

'gosto mais de vaca preta  coca cola com sorvete  do que de milk shake me prometo uma coca cola gelada para o  fim do dia essa coca cola gelada fez meu dia a combinação de sushi e cocacola é uma maravilha\n\nfelizmente o sushi nem tá perto de acabar miltonneves já chegou coca cola em muzambinho deixei a coca na geladeira e virou uma espécie de picolé pena que ficou preso na garrafa meu ser precisa de uma coca cola agora matei a última coca cola que tinha na geladeira 😊 big mac com coca cola eu mereço num dia quente assim quero me entupir de cheetos e coca cola eu sou apaixonada em coca cola 😍😍😍😍😰 tu representa com a coca eu represento com a pipoca cola p assistir how i met you mother cmg vamos depois da aula tomar coca cola com destilados no bar do zé coca cola de café é mt bom agora vai minha mãe comprou bala de coca cola acabei com tudo em um dia '

In [497]:
string_nao_feliz = ''
nao_feliz = nao_feliz.reset_index(drop=True)

for i in range(len(nao_feliz['Texto'])):
    string_nao_feliz += nao_feliz['Texto'][i] + ' '

string_nao_feliz

'acho que comer hamburguer de soja e se entupir de coca cola é ser hipócrita prometo parar com doce refrigerante coca cola etc maior bronca da nutricionista tomei tanta coca cola que zoei meu rim esse ritmo de 6l de coca cola por dia me deixou com diabetes coca cola é uma porcaria açucarada vai se f a coca cola patrocina esportes para nos fazer achar que engordamos porque não fazemos exercício fui na lanchonete da academia e só tinha coca cola a última coisa que eu precisava o gênio do meu irmão deixou a coca cola fora da geladeira afff sigo viciado em coca cola queria tanto sair do vício de tomar coca colamas esse mês vou parar a coca cola não matou minha sede '

<div id="q2_b"></div>

---

**b)** Apresente as tabelas que são equivalentes a $P(palavra|FELIZ)$ e $P(palavra|NAO\_FELIZ)$, e que contém todas as palavras presentes nas respectivas categorias.

Você deve guardá-las em variáveis e exibi-las de forma semelhante ao que foi feito com o comando `tweets.head()` acima.

**R.:**

$P(palavra|FELIZ)$

In [498]:
# tabela_relativa_feliz = string_feliz.split().value_counts(True)
string_feliz = string_feliz.split()
feliz_tabela = pd.Series(string_feliz)

feliz_tabela_relativa = feliz_tabela.value_counts(normalize=True)

$P(palavra|NAO\_FELIZ)$

In [499]:
string_nao_feliz = string_nao_feliz.split()
nao_feliz_tabela = pd.Series(string_nao_feliz)

nao_feliz_tabela_relativa = nao_feliz_tabela.value_counts(normalize=True)

<div id="q2_c"></div>

---

**c)** Considerando que: 
* Não deve ser feito nenhum tratamento para o caso de palavras inéditas
* Não devem ser usadas bibliotecas de *machine learning* ou estatística
* Atenha-se ao `pandas` e `numpy`



Por favor faça:

Classifique usando um classificador *Naive Bayes* se a frase `"prometo que vai ser a última coca cola do dia"` é uma frase que pertence à categoria `FELIZ` ou `NAO_FELIZ`

Deixe claro (pode ser em comentários) o significado de seus cálculos e a qual conclusão chegou. Principalmente deixei claro *em qual número* baseou a conclusão 

Atente que vai ser necessário calcular os valores de $P(FELIZ)$ e $P(NAO\_FELIZ)$




In [500]:
frase = "prometo que vai ser a última coca cola do dia" # frase a classificar

In [501]:
P_FELIZ = tweets['Categoria'].value_counts()['FELIZ'] / len(tweets)
P_NAO_FELIZ = tweets['Categoria'].value_counts()['NAO_FELIZ'] / len(tweets)

In [502]:
frase = frase.split()

probFraseDadoFeliz = 1
probFraseDadoNaoFeliz = 1

for palavra in frase:
    probFraseDadoFeliz *= feliz_tabela_relativa[palavra]
    probFraseDadoNaoFeliz *= nao_feliz_tabela_relativa[palavra]

In [503]:
probFelizDadoFrase = probFraseDadoFeliz * P_FELIZ
probNaoFelizDadoFrase = probFraseDadoNaoFeliz * P_NAO_FELIZ

print(probFelizDadoFrase)
print(probNaoFelizDadoFrase)

8.840985947103174e-19
1.4765785305458623e-18
